In [1]:
from bs4 import BeautifulSoup
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from operator import itemgetter
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy
import re
import requests
import statistics
import string
import json


In [2]:
dataset=numpy.load("backup.npy")

In [3]:
from nltk.corpus import stopwords
cachedStopWords = (stopwords.words("english"))

In [4]:
from nltk.stem.lancaster import LancasterStemmer

In [5]:
st = LancasterStemmer()

In [6]:
dataset=list(dataset)

In [7]:
ridotto=(dataset[0:10])

In [8]:
# take a list of song list
song_list=[]
for ab in ridotto:
    so=[]
    so.append(ab["lyrics"])
    song_list.append(so)

In [9]:
# take id value
prova=[]
for i in ridotto:
    prima=[]
    for k in (i['_id'].values()):
        prima.append(k)
    prova.append(prima)

In [10]:
# remove stopwords and punctuation
vuota=[]
for i in song_list:
    for line in i:
        prima=[]
        text=' '.join([word for word in line.lower().translate(str.maketrans('', '', string.punctuation)).split()
                      if len(word) >=2 and word not in cachedStopWords])
        prima.append(text)
        vuota.append(prima)

In [11]:
# find stem
vuotaa=[]
for i in vuota:
    documents = [[st.stem(word) for word in sentence.split(" ")] for sentence in i]
    vuotaa.append(documents)

In [12]:
#create new dictionaries with id and stemmed list of song
dizionario=[]
for i,j in zip(prova,vuotaa):
    riprova=dict(zip(i,j))
    dizionario.append(riprova)

In [13]:
frequenze=[]
for i in vuotaa:
    for j in i:
        common=Counter(j)
        frequenze.append(common)

In [14]:
for i in frequenze:
    factor=1.0/sum(i.values())
    for k in i:
        i[k] = i[k]*factor

In [15]:
ook=[]
for i in frequenze:
    ok=[]
    a=dict(i)
    ok.append(a)
    ook.append(ok)

In [16]:
#create new dictionaries with id and stemmed list of song normalized
dizionario=[]
for i,j in zip(prova,ook):
    riprova=dict(zip(i,j))
    dizionario.append(riprova)

In [17]:
#find all words used
parole=[]
for i in dizionario:
    for j in i.values():
        for k in j:
            parole.append(k)

In [18]:
#delete duplicate
parole = list(set(parole))

In [19]:
#just for test
parole_ridotto=['im', 'burn']

In [20]:
vocabulary=[]
for i in parole_ridotto:
    parole_cambiato= "term_id: " + i
    vocabulary.append(parole_cambiato)

In [21]:
vocabulary

['term_id: im', 'term_id: burn']

In [70]:
vocabulary={}
for i in parole_ridotto:
    vocabulary["term_id"]=i

In [71]:
(vocabulary)

{'term_id': 'burn'}

In [100]:
vocabulary=defaultdict(list)
for j in range(100):
    for i in parole_ridotto:
    
        vocabulary[j].append(i)
    

In [101]:
vocabulary=dict(vocabulary)

In [83]:
for i in vocabulary:
    print(i.values())

AttributeError: 'str' object has no attribute 'values'

In [90]:
any ("im") in vocabulary.values()

False

In [22]:
caricare=[]
for words in parole_ridotto:
    for i in dizionario:
        for j in i.values():
            
            if words in j:
                nuovo_dizionario=defaultdict(list)
                nuovo_dizionario[words].append((*i.keys(),(words,j[words])))
                caricare.append(dict(nuovo_dizionario))

In [23]:
output = {}
for d in caricare:
    for k,v in d.items():
        output.setdefault(k, []).append(v)
output=[{k:v} for k,v in output.items()]

In [24]:
output

[{'im': [[('5a1954dabd966f6888db3c79', ('im', 0.06451612903225806))],
   [('5a1954dabd966f6888db3c7d', ('im', 0.058823529411764705))],
   [('5a1954dabd966f6888db3ca3', ('im', 0.0625))],
   [('5a1954dabd966f6888db3ca4', ('im', 0.04672897196261682))],
   [('5a1954dabd966f6888db3ca5', ('im', 0.1509433962264151))],
   [('5a1954dabd966f6888db3ca6', ('im', 0.09836065573770492))],
   [('5a1954dabd966f6888db3b3c', ('im', 0.08012326656394453))],
   [('5a1954dabd966f6888db3b7d', ('im', 0.030303030303030304))]]},
 {'burn': [[('5a1954dabd966f6888db3c79', ('burn', 0.03225806451612903))]]}]

In [190]:
params = {'apiKey': 'jJFk7bsNFjagIf9nvxRQzq4AhVot1kkK'}
dbname = 'prova' #my db name in mlab
collection = 'terza_parte' #new collection created in mlab

In [191]:
url = 'https://api.mlab.com/api/1/databases/' + dbname + '/collections/' + collection
headers = {'content-type': 'application/json'}
data = json.dumps(output)
response = requests.post(url, data=data, params=params, headers=headers)